In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load DialoGPT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Initialize chat history
chat_history_ids = None

# Chat for 5 turns
for step in range(5):
    # Take user input
    text = input(">> You: ")

    # Encode the input and add EOS token
    input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors="pt")

    # Concatenate new user input with chat history (if there is one)
    bot_input_ids = torch.cat([chat_history_ids, input_ids], dim=-1) if chat_history_ids is not None else input_ids

    # Generate bot response with several improvements
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,                  # Max length of the conversation
        num_beams=5,                      # Beam search for more accurate responses
        top_p=0.9,                        # Nucleus sampling for better diversity and relevance
        temperature=0.7,                  # Reduce randomness for more coherent answers
        no_repeat_ngram_size=2,           # Avoid repetition of n-grams
        do_sample=True,                   # Use sampling instead of greedy search
        pad_token_id=tokenizer.eos_token_id  # Padding token
    )

    # Decode the generated response and remove special tokens
    output = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Print the model's response
    print(f"DialoGPT: {output}")

    # Update chat history to include this conversation turn
    chat_history_ids = chat_history_ids[:, -1000:]  # Optionally truncate history to avoid model input overflow


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

>> You: hello! i want to be rich


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


DialoGPT: Hello! I am rich!
>> You: no i want to be rich
DialoGPT: No you don't.
>> You: why?
DialoGPT: Why not?
>> You: can you tell me way how to be rich?
DialoGPT: I can't, but I can tell you how you can be a rich person
>> You: yes tell me
DialoGPT: tell me how i can become rich and how can i be one of the richest people in the world
